<a href="https://colab.research.google.com/github/RamDorak/AttendanceWeb/blob/main/SmartEnergyMeter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ML Linear Regression model to predict power consumption

In [ ]:
import requests
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

# Define the ThingSpeak API endpoint and the parameters
channel_id = 'YOUR_CHANNEL_ID'
read_key = 'YOUR_READ_API_KEY'
url = f'https://api.thingspeak.com/channels/{channel_id}/feeds.json'
params = {'api_key': read_key, 'results': 100}

# Send a GET request to retrieve the data from ThingSpeak
response = requests.get(url, params=params)

# Convert the JSON response to a Pandas dataframe
data = pd.json_normalize(response.json()['feeds'])

# Select the fields of interest
df = data[['field1', 'field2', 'field3']]

# Rename the columns
df.columns = ['time', 'voltage', 'current']

# Convert the data types
df = df.astype(float)

# Calculate the power consumption
df['power'] = df['voltage'] * df['current']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df[['voltage', 'current']], df['power'], test_size=0.2, random_state=0)

# Train the linear regression model
regressor = LinearRegression()
regressor.fit(X_train, y_train)

# Evaluate the model
score = regressor.score(X_test, y_test)
print('R-squared score:', score)

# Make a prediction using the latest voltage and current data
latest_voltage = df['voltage'].iloc[-1]
latest_current = df['current'].iloc[-1]
predicted_power = regressor.predict([[latest_voltage, latest_current]])
print('Predicted power consumption:', predicted_power[0])


ML Time Series Analysis model to visualise actual and predicted power consumption

In [ ]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error

# Retrieve data from ThingSpeak API endpoint
url = 'https://api.thingspeak.com/channels/YOUR_CHANNEL_ID/feeds.json'
params = {'api_key': 'YOUR_READ_API_KEY', 'results': 1000}
response = requests.get(url, params=params)
data = pd.json_normalize(response.json()['feeds'])

# Preprocess the data
data['created_at'] = pd.to_datetime(data['created_at'])
data.set_index('created_at', inplace=True)
data_resampled = data.resample('15T').mean().interpolate(method='linear')
power_data = data_resampled['field3']

# Split the data into training and testing sets
train_size = int(len(power_data) * 0.8)
train_data = power_data[:train_size]
test_data = power_data[train_size:]

# Train the ARIMA model
model = ARIMA(train_data, order=(1, 1, 1))
model_fit = model.fit()

# Make predictions on the testing data
predictions = model_fit.forecast(steps=len(test_data))[0]

# Evaluate the performance of the model
mse = mean_squared_error(test_data, predictions)
rmse = np.sqrt(mse)
print('Root Mean Squared Error:', rmse)

# Visualize the results
plt.plot(test_data)
plt.plot(predictions, color='red')
plt.show()
